In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('/kaggle/input/flightdata/flightdata.csv')
df.head()

FileNotFoundError: ignored

In [ ]:
df.shape

In [ ]:
df.isnull().values.any()

In [ ]:
df.isna().sum()

In [ ]:
df = df.drop('Unnamed: 25', axis=1)
df.isnull().sum()

In [ ]:
df = df[["MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", "ORIGIN", "DEST", "CRS_DEP_TIME", "ARR_DEL15"]]
df.isnull().sum()

In [ ]:
df[df.isnull().values.any(axis=1)].head()

In [ ]:
df = df.fillna({'ARR_DEL15': 1})
df.iloc[177:185]

In [ ]:
df.head()

In [ ]:
import math

for index, row in df.iterrows():
    df.loc[index, 'CRS_DEP_TIME'] = math.floor(row['CRS_DEP_TIME'] / 100)
df.head()

In [ ]:
df_1 = pd.get_dummies(df, columns=['ORIGIN', 'DEST'])
df_1.head()

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_1.drop('ARR_DEL15', axis=1), 
df_1['ARR_DEL15'], test_size=0.2, random_state=42)

In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifierr(random_state=13, class_weight = 'balanced_subsample')
#from imblearn.ensemble import BalancedRandomForestClassifier
#model = BalancedRandomForestClassifier(random_state=13,n_estimators=10)
model.fit(train_x, train_y)

In [ ]:
predicted = model.predict(test_x)
model.score(test_x, test_y)

In [ ]:
from sklearn.metrics import roc_auc_score
probabilities = model.predict_proba(test_x)

In [ ]:
roc_auc_score(test_y, probabilities[:, 1])

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, predicted)

In [ ]:
from sklearn import metrics
disp = metrics.plot_confusion_matrix(model, test_x, 
test_y,values_format = 'n',
display_labels= ["don't linger", "delayed"])

In [ ]:
from sklearn.metrics import precision_score

train_predictions = model.predict(train_x)
precision_score(train_y, train_predictions)

In [ ]:
from sklearn.metrics import recall_score

recall_score(train_y, train_predictions)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(test_y, probabilities[:, 1])
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], color='grey', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
metrics.auc(fpr, tpr)

In [ ]:
def predict_delay(departure_date_time, origin, destination):
    from datetime import datetime

    try:
        departure_date_time_parsed = datetime.strptime(departure_date_time, '%d/%m/%Y %H:%M:%S')
    except ValueError as e:
        return 'Error parsing date/time - {}'.format(e)

    month = departure_date_time_parsed.month
    day = departure_date_time_parsed.day
    day_of_week = departure_date_time_parsed.isoweekday()
    hour = departure_date_time_parsed.hour

    origin = origin.upper()
    destination = destination.upper()

    input = [{'MONTH': month,
              'DAY': day,
              'DAY_OF_WEEK': day_of_week,
              'CRS_DEP_TIME': hour,
              'ORIGIN_ATL': 1 if origin == 'ATL' else 0,
              'ORIGIN_DTW': 1 if origin == 'DTW' else 0,
              'ORIGIN_JFK': 1 if origin == 'JFK' else 0,
              'ORIGIN_MSP': 1 if origin == 'MSP' else 0,
              'ORIGIN_SEA': 1 if origin == 'SEA' else 0,
              'DEST_ATL': 1 if destination == 'ATL' else 0,
              'DEST_DTW': 1 if destination == 'DTW' else 0,
              'DEST_JFK': 1 if destination == 'JFK' else 0,
              'DEST_MSP': 1 if destination == 'MSP' else 0,
              'DEST_SEA': 1 if destination == 'SEA' else 0 }]

    return model.predict_proba(pd.DataFrame(input))[0][0]

In [ ]:
predict_delay('1/10/2018 21:45:00', 'JFK', 'ATL')

In [ ]:
predict_delay('2/10/2018 21:45:00', 'JFK', 'ATL')

In [ ]:
predict_delay('2/10/2018 21:45:00', 'JFK', 'ATL')

In [ ]:
predict_delay('2/10/2018 10:00:00', 'ATL', 'SEA')

In [ ]:
import numpy as np

labels = ('Oct 1', 'Oct 2', 'Oct 3', 'Oct 4', 'Oct 5', 'Oct 6', 'Oct 7')
values = (predict_delay('1/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('2/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('3/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('4/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('5/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('6/10/2018 21:45:00', 'JFK', 'ATL'),
          predict_delay('7/10/2018 21:45:00', 'JFK', 'ATL'))
alabels = np.arange(len(labels))

plt.bar(alabels, values, align='center', alpha=0.5)
plt.xticks(alabels, labels)
plt.ylabel('Probability of On-Time Arrival')
plt.ylim((0.0, 1.0))